- preprocess, dataset 분할 통한 평가 검증
- 결측치 채울 때 최소 1컬럼은 머신러닝
- 배포와 서비스(최소 2개 범주형 컬럼 포함)

In [38]:
# 1. 필요한 라이브러리 불러오기
import sklearn as sk
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import pickle

In [39]:
# 데이터 불러오기 
titanic = pd.read_csv('../../datasets/titanic_disaster_train.csv')
titanic.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [40]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [41]:
df_titanic = pd.DataFrame(titanic)
df_titanic.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [42]:
# 고유값의 수로 연속형과 범주형 구분하기
def category_columns(df_titanic):
    continuous_columns = []
    categorical_columns = []
    
    for column in df_titanic.columns :
        unique_values = df_titanic[column].nunique() # 고유값의 개수
        if unique_values < len(df_titanic) * 0.05 :  # 고유값 데이터 5% 미만시 범주형 간주
            categorical_columns.append(column)
        else :
            continuous_columns.append(column)
        
    return continuous_columns, categorical_columns

continuous, categorical = category_columns(df_titanic)

In [43]:
# 연속형 확인
continuous

['PassengerId', 'Name', 'Age', 'Ticket', 'Fare', 'Cabin']

In [44]:
# 범주형 확인
categorical

['Survived', 'Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']

In [45]:
# 데이터 결측치 확인
titanic[[
        'PassengerId', 'Name', 'Age', 'Ticket', 'Fare'
         , 'Cabin','Survived', 'Pclass', 'Sex', 'SibSp'
         , 'Parch', 'Embarked'
        ]].isna().sum()

PassengerId      0
Name             0
Age            177
Ticket           0
Fare             0
Cabin          687
Survived         0
Pclass           0
Sex              0
SibSp            0
Parch            0
Embarked         2
dtype: int64

In [51]:
# titanic['Embarked'].unique() # 'Embarked'의 unique 값 : 'S', 'C', 'Q', nan
# titanic['Cabin'].unique()

In [47]:
# 결측치 처리

# 평균값으로 빈칸 채움
titanic['Age'] = titanic['Age'].fillna(titanic['Age'].mean())

# Embarked 는 범주형임으로 최빈값으로 채움
titanic['Embarked'] = titanic['Embarked'].fillna(titanic['Embarked'].mode()[0])

# Cabin 값은 범주형임으로 최빈값으로 채움
titanic['Cabin'] = titanic['Cabin'].fillna(titanic['Cabin'].mode())[0]

In [50]:
# 결측치 처리 후 다시확인 
titanic.isnull().sum() 

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64